## Imports

In [ ]:
from typing import List
import re
import pandas as pd
import msticpy as mp
from bokeh.io import output_notebook
output_notebook()

mp.init_notebook()
pd.options.display.width = 0
pd.options.display.max_columns = 0
pd.options.display.max_colwidth = 0



## Lists

In [ ]:
LOLBAS = [
"adplus.exe",
"advpack.dll",
"agentexecutor.exe",
"appvlp.exe",
"aspnet_compiler.exe",
"at.exe",
"atbroker.exe",
"bash.exe",
"bginfo.exe",
"bitsadmin.exe",
"cdb.exe",
"certoc.exe",
"certreq.exe",
"certutil.exe",
"cmd.exe",
"cmdkey.exe",
"cmdl32.exe",
"cmstp.exe",
"comsvcs.dll",
"configsecuritypolicy.exe",
"control.exe",
"coregen.exe",
"csc.exe",
"cscript.exe",
"csi.exe",
"datasvcutil.exe",
"defaultpack.exe",
"desktopimgdownldr.exe",
"devtoolslauncher.exe",
"dfshim.dll",
"dfsvc.exe",
"diantz.exe",
"diskshadow.exe",
"dllhost.exe",
"dnscmd.exe",
"dnx.exe",
"dotnet.exe",
"dxcap.exe",
"esentutl.exe",
"eventvwr.exe",
"excel.exe",
"executable nameappinstaller.exe",
"expand.exe",
"explorer.exe",
"extexport.exe",
"extrac32.exe",
"findstr.exe",
"finger.exe",
"fltmc.exe",
"forfiles.exe",
"fsi.exe",
"fsianycpu.exe",
"ftp.exe",
"gfxdownloadwrapper.exe",
"gpscript.exe",
"hh.exe",
"ie4uinit.exe",
"ieadvpack.dll",
"ieaframe.dll",
"ieexec.exe",
"ilasm.exe",
"imewdbld.exe",
"infdefaultinstall.exe",
"installutil.exe",
"jsc.exe",
"makecab.exe",
"mavinject.exe",
"mftrace.exe",
"microsoft.workflow.compiler.exe",
"mmc.exe",
"mpcmdrun.exe",
"msbuild.exe",
"msconfig.exe",
"msdeploy.exe",
"msdt.exe",
"mshta.exe",
"mshtml.dll",
"msiexec.exe",
"msxsl.exe",
"netsh.exe",
"ntdsutil.exe",
"odbcconf.exe",
"offlinescannershell.exe",
"onedrivestandaloneupdater.exe",
"pcalua.exe",
"pcwrun.exe",
"pcwutl.dll",
"pktmon.exe",
"pnputil.exe",
"powerpnt.exe",
"powershell.exe",
"presentationhost.exe",
"print.exe",
"printbrm.exe",
"procdump(64).exe",
"psr.exe",
"rasautou.exe",
"rcsi.exe",
"reg.exe",
"regasm.exe",
"regedit.exe",
"regini.exe",
"register-cimprovider.exe",
"regsvcs.exe",
"regsvr32.exe",
"remote.exe",
"replace.exe",
"rpcping.exe",
"rundll32.exe",
"runonce.exe",
"runscripthelper.exe",
"sc.exe",
"schtasks.exe",
"scriptrunner.exe",
"settingsynchost.exe",
"setupapi.dll",
"shdocvw.dll",
"shell32.dll",
"sqldumper.exe",
"sqlps.exe",
"sqltoolsps.exe",
"squirrel.exe",
"stordiag.exe",
"syncappvpublishingserver.exe",
"syssetup.dll",
"te.exe",
"tracker.exe",
"ttdinject.exe",
"tttracer.exe",
"update.exe",
"url.dll",
"vbc.exe",
"verclsid.exe",
"visualuiaverifynative.exe",
"vsiisexelauncher.exe",
"vsjitdebugger.exe",
"wab.exe",
"wfc.exe",
"winword.exe",
"wmic.exe",
"workfolders.exe",
"wscript.exe",
"wsl.exe",
"wsreset.exe",
"wuauclt.exe",
"xwizard.exe",
"zipfldr.dll",
]

RECOON = [
  "chcp",
  "dsquery",
  "hostname",
  "ipconfig",
  "net",
  "netstat",
  "nltest",
  "whoami",
  "ldifde",
  "wmic",
  ]

CS_SPAWN_TO = [
"dllhost",
"gpupdate",
"mstsc",
"regsvr32",
"rundll32",
"svchost",
"userinit",
"werfault",
"wuauclt",
"wusa",
]

CHARACTERS = [
"—",
"―",
"˪",
"∕",
"⁄",
"ʰ",
"ʱ",
"ʲ",
"ˡ",
"ʳ",
"ʴ",
"ʵ",
"ʶ",
"ˢ", 
"ʷ",
"ˣ",
"ʸ",
"�",
re.escape("^"),
]

CS_SPAWN_TO = [ 
"dllhost",
"gpupdate",
"mstsc",
"regsvr32",
"rundll32",
"svchost",
"userinit",
"werfault",
"wuauclt",
"wusa",
]

COMMON_MALWARE_PATHS = [ 
"appdata",
"temp",
"perflogs",
"users",
]

OFFICE = [
    "winword.exe",
    "powerpnt.exe",
    "excel.exe",
    "outlook.exe",
    "onenote.exe",
]

# Host Investigation Notebook - Public

In [ ]:
mde_prov = mp.QueryProvider("M365D")
mde_prov.connect()

## LOLBAS Process Execution
### Generic Warning Flags
* User Writable Paths
* Suspicious Parent-Child Process Relationships
* Suspicious commandline for LOLBAS based off of https://lolbas-project.github.io/#
* Obfuscation Characters

In [ ]:
lolbas = mde_prov.mstical_host_triage.host_processes_query(start="-7", processes=LOLBAS, hostname="")

In [ ]:
lolbas["DeviceName"].unique()

### Look at and group all cmd

In [ ]:
lolbas_cmd = lolbas[lolbas["FileName"].str.contains("cmd")].groupby(["InitiatingProcessFileName", "FileName"]).agg(ProcessCommandLine=("ProcessCommandLine", set), count=("ProcessCommandLine", "count"))
lolbas_cmd

### look at and group all rundll

In [ ]:
lolbas_rundll = lolbas[lolbas["FileName"].str.contains("rundll")].groupby(["InitiatingProcessFileName", "FileName"]).agg(ProcessCommandLine=("ProcessCommandLine", set), count=("ProcessCommandLine", "count"))
lolbas_rundll

### Function to identify suspicious rundll

In [ ]:
def suspicious_lolbas_rundll32_execution(
    dataframe: pd.DataFrame,
    process_column: str = "FileName",
    process_name: str = "rundll32.exe",
    command_line_column: str = "ProcessCommandLine",
    suspicious_params: List[str] = [
        "appdata"
        ".scr",
        "advpack.dll",
        "cmd",
        "Control_RunDLL",
        "comsvcs",
        "cscript",
        "desk.cpl",
        "dfshim.dll",
        "FileProtocolHandler",
        "ieadvpack.dll",
        "ieframe.dll",
        "InstallHinfSection",
        "javascript:",
        "LaunchApplication",
        "LaunchINFSection",
        "mshtml.dll",
        "OpenURL",
        "pcwutl.dll",
        "powershell",
        "PrintHTML",
        "RegisterOCX",
        "RegisterXLL",
        "RegisterXL",
        "RouteTheCall",
        "setupapi.dll",
        "setupInfObjectInstallAction",
        "shdocvw.dll",
        "shell32.dll",
        "ShellExec_RunDLL",
        "ShOpenVerbApplication",
        "ShOpenVerbShortcut",
        "syssetup.dll",
        "temp"
        "url.dll",
        "users",
        "wscript",
        "zipfldr.dll",
    ],
    **kwargs,
) -> pd.DataFrame:
    """Filter on processes that contain a suspicious file name."""
    del kwargs
    return dataframe[
        (dataframe[process_column].str.casefold() == process_name.casefold())
        & dataframe[command_line_column].str.contains("|".join(suspicious_params))
    ]

In [ ]:
suspicious_lolbas_rundll32_execution(lolbas).groupby(["InitiatingProcessFileName", "FileName"]).agg(ProcessCommandLine=("ProcessCommandLine", set), count=("ProcessCommandLine", "count"))

### Look at and group all PowerShell

In [ ]:
lolbas_powershell = lolbas[lolbas["FileName"].str.contains("powershell")].groupby(["InitiatingProcessFileName", "FileName"]).agg(ProcessCommandLine=("ProcessCommandLine", set), count=("ProcessCommandLine", "count"))
lolbas_powershell

### Obfuscation Characters

In [ ]:
lolbas_obfuscation = lolbas[lolbas["ProcessCommandLine"].str.contains("|".join(CHARACTERS))].groupby(["InitiatingProcessFileName", "FileName"]).agg(ProcessCommandLine=("ProcessCommandLine", set), count=("ProcessCommandLine", "count"))
lolbas_obfuscation

### Group everything

In [ ]:
cmd_lines = ["GetDeviceTicket", "jimbob","UpdateDeploy.dll",
"UpdateDeploymentProvider.dll","Windows Defender Advanced Threat Protection",
"Get-WindowsFeature -Name Web-Server", "-RestrictPrivileges -AccessKey",
"/noconfig /fullpaths"]
cmd_lines = [f"(?:{cmd_line})" for cmd_line in map(str, cmd_lines)]
search_pattern = "|".join(cmd_lines)

lolbas_exclude = lolbas[~lolbas["ProcessCommandLine"].str.contains(search_pattern)]
lolbas_exclude.groupby(["InitiatingProcessFileName", "FileName"]).agg(ProcessCommandLine=("ProcessCommandLine", set), count=("ProcessCommandLine", "count"))

## LOLBAS Parent Process Execution
### Generic Warning Flags
* User Writable Paths
* Suspicious Parent-Child Process Relationships
* Suspicious commandline for LOLBAS based off of https://lolbas-project.github.io/#
* Obfuscation Characters

In [ ]:
lolbas_parent = mde_prov.mstical_host_triage.host_parent_processes_query(start="-7", parent_processes=LOLBAS, hostname="")

In [ ]:
cmd_lines = ["AccessKey","OUT:C:\\\Windows\\\TEMP\\\RES","cvtres.exe /NOLOGO /READONLY", "/get /subcategory", "noconfig /fullpaths"]
cmd_lines = [f"(?:{cmd_line})" for cmd_line in map(str, cmd_lines)]
search_pattern = "|".join(cmd_lines)

lolbas_parent_exclude = lolbas_parent[~lolbas_parent["ProcessCommandLine"].str.contains(search_pattern)]

lolbas_parent_exclude.groupby(["InitiatingProcessFileName", "FileName"]).agg(ProcessCommandLine=("ProcessCommandLine", set), count=("ProcessCommandLine", "count"))

## CS Spawn-To Processes
### Generic Warning Flags
* User Writable Paths
* Suspicious Parent-Child Process Relationships
* Suspicious commandline for LOLBAS based off of https://lolbas-project.github.io/#
* Obfuscation Characters

In [ ]:
cs_spawn_to = mde_prov.mstical_host_triage.host_parent_processes_query(start="-7", parent_processes=CS_SPAWN_TO,hostname="")

In [ ]:
cs_spawn_to.groupby(["InitiatingProcessFileName", "FileName"]).agg(ProcessCommandLine=("ProcessCommandLine", set), count=("ProcessCommandLine", "count"))

## Child Processes Of Office Applications
### Generic Warning Flags
* Suspicious Parent-Child Process Relationships
* Suspicious commandline for LOLBAS based off of https://lolbas-project.github.io/#

In [ ]:
office_child_process = mde_prov.mstical_host_triage.host_parent_processes_query(start="-7", parent_processes=OFFICE, hostname="")

In [ ]:
office_child_process

## Common Interpreters
* powershell
* cmd
* wscript
* cscript

### Generic Warning Flags
* User Writable Paths
* Suspicious Parent-Child Process Relationships
* Suspicious commandline strings for powershell/cmd
* Obfuscation Characters

In [ ]:
common_interpreter = mde_prov.mstical_host_triage.host_processes_query(start="-7", processes=["powershell", "cmd.exe", "wscript.exe", "cscript.exe"], hostname="")
common_interpreter

In [ ]:
common_interpreter["FileName"].unique()

In [ ]:
common_interpreter[common_interpreter["ProcessCommandLine"].str.contains("cscript")]["ProcessCommandLine"].unique()

In [ ]:
cmd_lines = ["SenseIR","gc_worker"]
cmd_lines = [f"(?:{cmd_line})" for cmd_line in map(str, cmd_lines)]
search_pattern = "|".join(cmd_lines)

common_interpreter_exclude = common_interpreter[~common_interpreter["InitiatingProcessFileName"].str.contains(search_pattern)]
common_interpreter_exclude.groupby(["InitiatingProcessFileName", "FileName"]).agg(ProcessCommandLine=("ProcessCommandLine", set), count=("ProcessCommandLine", "count"))

In [ ]:
common_interpreter_obfuscation = common_interpreter[common_interpreter["ProcessCommandLine"].str.contains("|".join(CHARACTERS))].groupby(["InitiatingProcessFileName", "FileName"]).agg(ProcessCommandLine=("ProcessCommandLine", set), count=("ProcessCommandLine", "count"))
common_interpreter_obfuscation

## Recon Activity
### Generic Warning Flags
* User Writable Paths
* Suspicious Parent-Child Process Relationships
* Suspicious commandline for the recon process
* Obfuscation Characters

In [ ]:
recon = mde_prov.mstical_host_triage.host_processes_query(start="-7", processes=RECOON, hostname="")
recon

In [ ]:
recon.groupby(["InitiatingProcessFileName", "FileName"]).agg(ProcessCommandLine=("ProcessCommandLine", set), count=("ProcessCommandLine", "count"))

## Loaded Modules - system.management.automation.dll
### Generic Warning Flags
* Suspicious commandline for the process
* Weird process filename or unordinary location
* Unusual processes loading the DLL e.g rundll32 or another popular spawn-to value, or office application

In [ ]:
powershell_module_load = mde_prov.mstical_host_triage.host_image_load_query(start="-7", loaded_image=["system.management.automation.dll"], hostname="")

In [ ]:
pd.DataFrame(powershell_module_load["InitiatingProcessFolderPath"].unique())

## Loaded Modules - clr.dll
### Generic Warning Flags
* Suspicious commandline for the process
* Weird process filename or unordinary location
* Unusual processes loading the DLL e.g rundll32 or another popular spawn-to value, or office application

In [ ]:
dotnet_module_load = mde_prov.mstical_host_triage.host_image_load_query(start="-7", loaded_image=["clr.dll"], hostname="")

In [ ]:
pd.DataFrame(dotnet_module_load["InitiatingProcessFolderPath"].unique())

## Loaded Modules - wmiutils.dll or fastprox.dll
### Generic Warning Flags
* Suspicious commandline for the process
* Weird process filename or unordinary location
* Unusual processes loading the DLL e.g rundll32 or another popular spawn-to value, or office application

In [ ]:
wmi_module_load = mde_prov.mstical_host_triage.host_image_load_query(start="-7", loaded_image=["wmiutils.dll", "fastprox.dll"], hostname="")

In [ ]:
pd.DataFrame(wmi_module_load["InitiatingProcessFolderPath"].unique())

In [ ]:
pd.DataFrame(wmi_module_load[wmi_module_load["InitiatingProcessCommandLine"].str.contains("powershell")]["InitiatingProcessCommandLine"].drop_duplicates())

## Loaded Modules - winhttp.dll
### Generic Warning Flags
* Suspicious commandline for the process
* Weird process filename or unordinary location
* Unusual processes loading the DLL e.g rundll32 or another popular spawn-to value
* a process you would not expect to be making outbound internet connections

In [ ]:
winhttp_module_load = mde_prov.mstical_host_triage.host_image_load_query(start="-7", loaded_image=["winhttp.dll"], hostname="")

In [ ]:
pd.DataFrame(winhttp_module_load["InitiatingProcessFolderPath"].unique())

## Loaded Modules - Microsoft.WSMan.Management.ni.dll, WsmSvc.dll, WsmAuto.dll
### Generic Warning Flags
* Suspicious commandline for the process
* Weird process filename or unordinary location
* Unusual processes loading the DLL e.g rundll32 or another popular spawn-to value
* a process you would not expect to be making outbound internet connections

In [ ]:
winrm_module_load = mde_prov.mstical_host_triage.host_image_load_query(start="-7", loaded_image=["Microsoft.WSMan.Management.ni.dll", "WsmSvc.dll", "WsmAuto.dll"], hostname="")

In [ ]:
pd.DataFrame(winrm_module_load["InitiatingProcessFolderPath"].unique())

## Loaded Modules - netapi32.dll
### Generic Warning Flags
* Suspicious commandline for the process
* Weird process filename or unordinary location
* Unusual processes loading the DLL e.g rundll32 or another popular spawn-to value
* a process you would not expect to be making outbound internet connections

In [ ]:
net_module_load = mde_prov.mstical_host_triage.host_image_load_query(start="-7", loaded_image=["netapi32.dll"], hostname="")

In [ ]:
pd.DataFrame(net_module_load["InitiatingProcessFolderPath"].unique())

## Registry Run Keys
### Generic Warning Flags
* Suspicious commandline for the key value
* Weird process filename or unordinary location, like appdata perflogs etc
* Weird key value name
* a lolbas process
* service not known to be normal in environment 

In [ ]:
asep_keys = mde_prov.mstical_host_triage.host_asep_registry_key_set_query(start="-7", hostname="")

In [ ]:
pd.DataFrame(asep_keys["RegistryKey"].unique())

## Scheduled Tasks
### Generic Warning Flags
* Suspicious commandline for the scheduled task
* Weird process filename or unordinary location, like appdata perflogs etc
* Weird scheduled task name
* a lolbas process
* service not known to be normal in environment 

In [ ]:
sch_tasks = mde_prov.mstical_host_triage.host_scheduled_task_creation_query(start="-7", hostname="")

In [ ]:
pd.DataFrame(sch_tasks["AdditionalFields"].unique())

## Services
### Generic Warning Flags
* Suspicious commandline for the service
* Weird process filename or unordinary location, like appdata perflogs etc
* Weird service name
* a lolbas process
* service not known to be normal in environment 

In [ ]:
services = mde_prov.mstical_host_triage.host_get_services_query(start="-7", hostname="")

## Process Injection
### Generic Warning Flags
* Suspicious commandline for the process
* Weird process filename or unordinary location
* Unusual processes conducting the injection, like a known spawnto or office application

In [ ]:
process_injection = mde_prov.mstical_host_triage.host_process_injection_api_calls_query(start="-7", hostname="")

In [ ]:
sus_process_injections = CS_SPAWN_TO + ["winword","excel","powerpnt","outlook","MetricsExtension.Native.exe"]
sus_process_injection = [f"(?:{sus_process_injection})" for sus_process_injection in map(str, sus_process_injections)]
search_pattern = "|".join(sus_process_injections)

process_injection_exclude = process_injection[process_injection["FileName"].str.contains(search_pattern)]

process_injection_exclude.groupby(["InitiatingProcessFileName", "FileName"]).agg(ProcessCommandLine=("ProcessCommandLine", set), count=("ProcessCommandLine", "count"))

In [ ]:
def suspicious_process_injection(
    dataframe: pd.DataFrame,
    filename_column: str = "FileName",
    suspicious_param: list[str] = CS_SPAWN_TO + ["winword","excel","powerpnt","outlook",],
    **kwargs,
) -> pd.DataFrame:
        """Filter on Suspicious Processes Conducting Process Injection"""
        del kwargs
        return dataframe[
            dataframe[filename_column].str.contains("|".join(sus_process_injection))
        ]


In [ ]:
suspicious_process_injection(process_injection)

## Named Pipes
### Generic Warning Flags
* Suspicious commandline for the process
* Weird process filename or unordinary location
* masquerading as legitimate process
* random string of characters

In [ ]:
named_pipes = mde_prov.mstical_host_triage.host_malicious_named_pipes_query(start="-7", hostname="")

## User Writable Paths
### Generic Warning Flags
* Suspicious commandline for the process
* Weird process filename or unordinary location
* masquerading as legitimate process
* random string of characters

In [ ]:
uw_path = mde_prov.mstical_host_triage.host_process_execution_from_path_query(start="-7", file_path=COMMON_MALWARE_PATHS, hostname="")

In [ ]:
uw_path.groupby(["FileName", "FolderPath"]).agg(ProcessCommandLine=("ProcessCommandLine", set), count=("ProcessCommandLine", "count"))